<a href="https://www.kaggle.com/code/theayushanand/shannon-s-index?scriptVersionId=114708175" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Shannon's Index with OBIS Data

## importing modules

In [1]:
try:
    from pyobis import occurrences
except:
    !pip install -q pyobis
    from pyobis import occurrences
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import math

## data

In [2]:
query = occurrences.search(
    geometry="POLYGON ((58.3301 19.0935, 69.8145 19.0381, 69.8145 9.5161, 58.6230 9.6316, 58.3301 19.0935))", # this is a geometry in the Arabian Sea (right of India)
)

In [3]:
query.api_url

'https://api.obis.org/v3/occurrence?geometry=POLYGON+%28%2858.3301+19.0935%2C+69.8145+19.0381%2C+69.8145+9.5161%2C+58.6230+9.6316%2C+58.3301+19.0935%29%29&offset=0&mof=False'

In [4]:
query.mapper_url

'https://mapper.obis.org/?geometry=POLYGON+%28%2858.3301+19.0935%2C+69.8145+19.0381%2C+69.8145+9.5161%2C+58.6230+9.6316%2C+58.3301+19.0935%29%29&offset=0&mof=False'

In [5]:
query.execute()

Fetching: [████████████████████████████████████████████████████████████████████████████████████████████████████] 64210/64210
Fetched 64210 records.


,country,brackish,date_year,scientificNameID,year,scientificName,absence,dropped,aphiaID,decimalLatitude,...,identificationID,locationRemarks,verbatimSRS,georeferenceVerificationStatus,previousIdentifications,georeferencedBy,minimumElevationInMeters,maximumElevationInMeters,georeferenceProtocol,islandGroup
0,SOVIET UNION,True,1980.0,urn:lsid:marinespecies.org:taxname:101,1980,Gastropoda,False,False,101,11.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UNITED STATES,NaN,1995.0,urn:lsid:marinespecies.org:taxname:345515,1995,Prochlorococcus,False,False,345515,17.199600,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,True,NaN,urn:lsid:marinespecies.org:taxname:1137,NaN,Cumacea,False,False,1137,12.030000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SOVIET UNION,NaN,1970.0,urn:lsid:marinespecies.org:taxname:534090,1970,Fiona,False,False,138007,10.100000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,True,1995.0,urn:lsid:marinespecies.org:taxname:393148,1995,Rhizobiales,False,False,393148,16.050667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64205,UNITED STATES,NaN,1995.0,urn:lsid:marinespecies.org:taxname:345515,1995,Prochlorococcus,False,False,345515,17.686200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64206,NaN,NaN,1997.0,urn:lsid:marinespecies.org:taxname:418106,1997,Globigerinella calida,False,False,418106,16.272300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64207,SOVIET UNION,True,1980.0,urn:lsid:marinespecies.org:taxname:1078,1980,Ostracoda,False,False,1078,17.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64208,NaN,NaN,1995.0,urn:lsid:marinespecies.org:taxname:113455,1995,Globigerinoides tenellus,False,False,418107,14.443400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df = query.data

## generating the shannon's index

### 1. for the whole geometry

In [7]:
df.dropna(subset=["species"]).groupby("species").id.count()

species
Abditodentrix pseudothalmanni     2
Abudefduf taurus                  2
Abylopsis eschscholtzii           1
Abylopsis tetragona              28
Acanthoica quattrospina           6
                                 ..
Xiphias gladius                   4
Zenkevitchiella atlantica         1
Zonosagitta bedoti               54
Zonosagitta pulchra              50
Zygosphaera hellenica             1
Name: id, Length: 1093, dtype: int64

In [8]:
sh_df = pd.DataFrame(columns=["species","number","coeff"])
sh_df["species"] = Out[7].index
sh_df["number"] = Out[7].values

In [9]:
sh_df

,species,number,coeff
0,Abditodentrix pseudothalmanni,2,NaN
1,Abudefduf taurus,2,NaN
2,Abylopsis eschscholtzii,1,NaN
3,Abylopsis tetragona,28,NaN
4,Acanthoica quattrospina,6,NaN
...,...,...,...
1088,Xiphias gladius,4,NaN
1089,Zenkevitchiella atlantica,1,NaN
1090,Zonosagitta bedoti,54,NaN
1091,Zonosagitta pulchra,50,NaN


In [10]:
sums = sh_df["number"].sum()
def shannon_coeff(x):
    pi = x/sums
    return pi*math.log(pi)

In [11]:
sh_df["coeff"] = sh_df["number"].apply(shannon_coeff)

In [12]:
sh_df

,species,number,coeff
0,Abditodentrix pseudothalmanni,2,-0.000732
1,Abudefduf taurus,2,-0.000732
2,Abylopsis eschscholtzii,1,-0.000393
3,Abylopsis tetragona,28,-0.007391
4,Acanthoica quattrospina,6,-0.001941
...,...,...,...
1088,Xiphias gladius,4,-0.001357
1089,Zenkevitchiella atlantica,1,-0.000393
1090,Zonosagitta bedoti,54,-0.012884
1091,Zonosagitta pulchra,50,-0.012078


Now let us compute the `diversity` by adding the coefficients.

In [13]:
-sh_df["coeff"].sum()

5.200478193507969

### 2. Shannon's index for different bins in the geometry
Let us do rectangular binning and round of the coordinates to 3 decimal places and plot on the map to look at the estimate shannon's index.

In [14]:
_df = df.dropna(subset=["species"])

In [15]:
_df = _df.round({"decimalLongitude":3, "decimalLatitude":3})

In [16]:
_sh_df = _df.groupby(["decimalLongitude", "decimalLatitude", "species"]).id.count()
_sh_df = pd.DataFrame(_sh_df)
_sh_df

id
decimalLongitude decimalLatitude species                         
58.360           18.66           Grampus griseus                1
58.380           18.77           Aidanosagitta neglecta         1
                                 Aidanosagitta regularis        1
                                 Alacia alata                   2
                                 Conchoecetta giesbrechti       2
...                                                            ..
69.733           16.90           Orbulina universa              1
                                 Pulleniatina obliquiloculata   1
                                 Siriella gracilis              2
                                 Trilobatus sacculifer          1
69.738           13.57           Carcharhinus falciformis       6

[6933 rows x 1 columns]

In [17]:
_sh_df["sum"] = _df.groupby(["decimalLongitude", "decimalLatitude"]).id.count()

In [18]:
_temp_df = _df.groupby(["decimalLongitude", "decimalLatitude"]).id.count()
_temp_df = pd.DataFrame(_temp_df)
_temp_df

,,id
decimalLongitude,decimalLatitude,
58.360,18.660,1
58.380,18.770,44
58.394,17.310,210
58.399,18.028,1
58.421,18.099,1
...,...,...
69.708,18.995,3
69.730,16.900,15
69.733,16.867,7


In [19]:
_sh_df

id  sum
decimalLongitude decimalLatitude species                              
58.360           18.66           Grampus griseus                1    1
58.380           18.77           Aidanosagitta neglecta         1   44
                                 Aidanosagitta regularis        1   44
                                 Alacia alata                   2   44
                                 Conchoecetta giesbrechti       2   44
...                                                            ..  ...
69.733           16.90           Orbulina universa              1   11
                                 Pulleniatina obliquiloculata   1   11
                                 Siriella gracilis              2   11
                                 Trilobatus sacculifer          1   11
69.738           13.57           Carcharhinus falciformis       6    6

[6933 rows x 2 columns]

In [20]:
_sh_df["pi"] = _sh_df["id"]/_sh_df["sum"]

In [21]:
_sh_df["coeff"] = -_sh_df["pi"]*_sh_df["pi"].apply(math.log)

In [22]:
_sh_df

id  sum  \
decimalLongitude decimalLatitude species                                 
58.360           18.66           Grampus griseus                1    1   
58.380           18.77           Aidanosagitta neglecta         1   44   
                                 Aidanosagitta regularis        1   44   
                                 Alacia alata                   2   44   
                                 Conchoecetta giesbrechti       2   44   
...                                                            ..  ...   
69.733           16.90           Orbulina universa              1   11   
                                 Pulleniatina obliquiloculata   1   11   
                                 Siriella gracilis              2   11   
                                 Trilobatus sacculifer          1   11   
69.738           13.57           Carcharhinus falciformis       6    6   

                                                                     pi  \
decimalLongitude decimalLatitude species                                  
58.360           18.66           Grampus griseus               1.000000   
58.380           18.77           Aidanosagitta neglecta        0.022727   
                                 Aidanosagitta regularis       0.022727   
                                 Alacia alata                  0.045455   
                                 Conchoecetta giesbrechti      0.045455   
...                                                                 ...   
69.733           16.90           Orbulina universa             0.090909   
                                 Pulleniatina obliquiloculata  0.090909   
                                 Siriella gracilis             0.181818   
                                 Trilobatus sacculifer         0.090909   
69.738           13.57           Carcharhinus falciformis      1.000000   

                                                                  coeff  
decimalLongitude decimalLatitude species                                 
58.360           18.66           Grampus griseus              -0.000000  
58.380           18.77           Aidanosagitta neglecta        0.086004  
                                 Aidanosagitta regularis       0.086004  
                                 Alacia alata                  0.140502  
                                 Conchoecetta giesbrechti      0.140502  
...                                                                 ...  
69.733           16.90           Orbulina universa             0.217990  
                                 Pulleniatina obliquiloculata  0.217990  
                                 Siriella gracilis             0.309954  
                                 Trilobatus sacculifer         0.217990  
69.738           13.57           Carcharhinus falciformis     -0.000000  

[6933 rows x 4 columns]

Now let us run a sum over the coefficients.

In [23]:
_sh_df.reset_index()

,decimalLongitude,decimalLatitude,species,id,sum,pi,coeff
0,58.360,18.66,Grampus griseus,1,1,1.000000,-0.000000
1,58.380,18.77,Aidanosagitta neglecta,1,44,0.022727,0.086004
2,58.380,18.77,Aidanosagitta regularis,1,44,0.022727,0.086004
3,58.380,18.77,Alacia alata,2,44,0.045455,0.140502
4,58.380,18.77,Conchoecetta giesbrechti,2,44,0.045455,0.140502
...,...,...,...,...,...,...,...
6928,69.733,16.90,Orbulina universa,1,11,0.090909,0.217990
6929,69.733,16.90,Pulleniatina obliquiloculata,1,11,0.090909,0.217990
6930,69.733,16.90,Siriella gracilis,2,11,0.181818,0.309954
6931,69.733,16.90,Trilobatus sacculifer,1,11,0.090909,0.217990


In [24]:
_temp_df = _sh_df.reset_index()

In [25]:
raw_data = pd.DataFrame(_temp_df.groupby(["decimalLongitude", "decimalLatitude"]).coeff.sum()).reset_index()

In [26]:
raw_data

,decimalLongitude,decimalLatitude,coeff
0,58.360,18.660,0.000000
1,58.380,18.770,3.437616
2,58.394,17.310,2.895264
3,58.399,18.028,0.000000
4,58.421,18.099,0.000000
...,...,...,...
823,69.708,18.995,1.098612
824,69.730,16.900,2.708050
825,69.733,16.867,1.945910
826,69.733,16.900,2.271869


This looks superb!

In [27]:
import plotly.express as px

In [28]:
len(raw_data["decimalLatitude"])== len(raw_data["decimalLongitude"])

True

In [29]:
data = [
    list(raw_data["decimalLongitude"]),
    list(raw_data["coeff"]),
    list(raw_data["decimalLatitude"]),
    
    
]

In [30]:
fig = px.imshow(
    data,
    labels=dict(x="longitude", y="latitude", color="H"),
)
fig.show()

### 3. putting everything into a function

In [31]:
def gen_shannon(df, decimals):
    if "id" not in df.columns:
        df.rename({"gbifID":"id"}, inplace=True)
    
    _df = df.dropna(subset=["species"]).round({"decimalLongitude":decimals, "decimalLatitude":decimals})[["decimalLongitude","decimalLatitude","species","id"]]
    _sh_df = pd.DataFrame(_df.groupby(["decimalLongitude", "decimalLatitude", "species"]).id.count())
    _sh_df["sum"] = _df.groupby(["decimalLongitude", "decimalLatitude"]).id.count()
    _sh_df["coeff"] = _sh_df["id"]/_sh_df["sum"]*(_sh_df["id"]/_sh_df["sum"]).apply(math.log)
    return _sh_df.reset_index().groupby(["decimalLongitude", "decimalLatitude"]).coeff.sum().reset_index()


In [32]:
gen_shannon(df, 3)

,decimalLongitude,decimalLatitude,coeff
0,58.360,18.660,0.000000
1,58.380,18.770,-3.437616
2,58.394,17.310,-2.895264
3,58.399,18.028,0.000000
4,58.421,18.099,0.000000
...,...,...,...
823,69.708,18.995,-1.098612
824,69.730,16.900,-2.708050
825,69.733,16.867,-1.945910
826,69.733,16.900,-2.271869


This is awesome! Whoo.